In [17]:
# re
import re

# data reading / cleaning
from gut_tokenize import read_data, preprocess

In [18]:
directory = "../Gutenberg/cleaned_texts/"
titles = []
texts = []
titles, texts = read_data(directory)

for title in titles:
    print(title)

robinson_crusoe_clean.txt
great_gatsby_clean.txt
around_the_world_in_80_days_clean.txt
germinal-clean.txt
pride_and_prejudice_clean.txt
the_jungle_clean.txt
scarlet_letter_clean.txt
great_expectations_clean.txt
moby-clean.txt
anna_karenina_clean.txt
twenty_thousand_leagues_under_the_sea_clean.txt
alice_in_wonderland_clean.txt
frankenstein_clean.txt
sense_and_sensibility_clean.txt
middlemarch_clean.txt
tom_jones_clean.txt
utopia_clean.txt
the_time_machine_clean.txt
north_and_south_clean.txt
the_fortune_of_the_rougons_clean.txt
adam_bede_clean.txt
babbit_clean.txt
the_last_man_clean.txt
madame_bovary_clean.txt
picture_of_dorian_grey_clean.txt
heart_of_darkness_clean.txt
crime_and_punishment_clean.txt
bleak_house_clean.txt
